In [1]:
import pandas as pd

In [2]:
EPS = 1

In [3]:
result_file = "test_result_w_succ_same_nodes_journ2.csv"
df = pd.read_csv(result_file)

df = df.rename(columns={"Dataset": "Graph", "Actual": "Rank"})

df.head()

,Graph,Rank,Predicted
0,star_graph_n7,0.0,0.000000
1,star_graph_n7,3.0,3.596601
2,star_graph_n7,3.0,3.596601
3,star_graph_n7,0.0,0.000000
4,star_graph_n7,3.0,2.976836


In [4]:
df['Difference'] = abs(df['Rank']-df['Predicted'])
df['Difference_sq'] = df['Difference']**2
# df.head()

In [5]:
df['EPS'] = df['Rank'].apply(lambda x: EPS if x==0.0 else 0.0)
# df

In [6]:
df['RE'] = df['Difference']/(df['Rank']+df['EPS'])
# df.head()

In [7]:
df

,Graph,Rank,Predicted,Difference,Difference_sq,EPS,RE
0,star_graph_n7,0.0,0.000000,0.000000,0.000000,1.0,0.000000
1,star_graph_n7,3.0,3.596601,0.596601,0.355933,0.0,0.198867
2,star_graph_n7,3.0,3.596601,0.596601,0.355933,0.0,0.198867
3,star_graph_n7,0.0,0.000000,0.000000,0.000000,1.0,0.000000
4,star_graph_n7,3.0,2.976836,0.023164,0.000537,0.0,0.007721
...,...,...,...,...,...,...,...
5413,graph_powerlaw_cluster_graph_n7,4.0,3.892483,0.107517,0.011560,0.0,0.026879
5414,graph_powerlaw_cluster_graph_n7,1.0,0.920226,0.079774,0.006364,0.0,0.079774
5415,graph_powerlaw_cluster_graph_n7,3.0,3.505972,0.505972,0.256008,0.0,0.168657
5416,graph_powerlaw_cluster_graph_n7,1.0,0.936819,0.063181,0.003992,0.0,0.063181


In [8]:
df['RE'].sum()/len(df)

0.06847626014918701

In [9]:
def group_by_table(df, groupby):
    values_count = df[groupby].value_counts().reset_index()
    values_count = values_count.rename(columns={"count": "Count"})
    re_sum = df.groupby(groupby)["RE"].sum().reset_index()
    mre_error = df.groupby(groupby)["RE"].mean().reset_index()
    mre_error = mre_error.rename(columns={"RE": "MRE"})
    mse_error = df.groupby(groupby)["Difference_sq"].mean().reset_index()
    mse_error = mse_error.rename(columns={"Difference_sq": "MSE"})

    return values_count, re_sum, mre_error, mse_error

# Group by Rank

In [10]:
groupby = ['Rank']

values_count, re_sum, mre_error, mse_error = group_by_table(df, groupby)

df_result = pd.merge(values_count, re_sum).sort_values(by=['Count', 'RE'], ascending=[False, False])
df_result = pd.merge(df_result, mre_error)
df_result = pd.merge(df_result, mse_error)
df_result.to_csv("test_analysis_grp_by_rank.csv", index=False)
df_result


,Rank,Count,RE,MRE,MSE
0,3.0,1726,151.821123,0.087961,0.128281
1,2.0,1616,124.583111,0.077094,0.047650
2,1.0,995,37.609068,0.037798,0.002378
3,4.0,676,50.706360,0.075009,0.135932
4,0.0,334,0.000000,0.000000,0.000000
5,5.0,67,5.836522,0.087112,0.335893
6,6.0,4,0.448195,0.112049,0.741834


# Group by Dataset

In [11]:
groupby = ['Graph']

values_count, re_sum, mre_error, mse_error = group_by_table(df, groupby)

df_result = pd.merge(values_count, re_sum).sort_values(by=['Count', 'RE'], ascending=[False, False])
df_result = pd.merge(df_result, mre_error)
df_result = pd.merge(df_result, mse_error)
df_result.to_csv("test_analysis_grp_by_dataset.csv", index=False)
df_result


,Graph,Count,RE,MRE,MSE
0,graph_random_regular_graph_n7_d4,3907,268.283650,0.068667,0.079988
1,graph_powerlaw_cluster_graph_n7,1350,97.185105,0.071989,0.073462
2,star_graph_5_n7,23,1.497408,0.065105,0.050951
3,star_graph_n7,23,1.177959,0.051216,0.058550
4,star_graph_4_n7,23,0.934693,0.040639,0.051482
5,star_graph_2_n7,23,0.602694,0.026204,0.039015
6,star_graph_6_n7,23,0.520716,0.022640,0.030334
7,star_graph_1_n7,23,0.417483,0.018151,0.027614
8,star_graph_3_n7,23,0.384669,0.016725,0.022778


# Group by Dataset W/O Rank 0

In [12]:
groupby = ['Graph']

df_wo_0 = df[df['Rank']!=0]
values_count, re_sum, mre_error, mse_error = group_by_table(df_wo_0, groupby)

df_result = pd.merge(values_count, re_sum).sort_values(by=['Count', 'RE'], ascending=[False, False])
df_result = pd.merge(df_result, mre_error)
df_result = pd.merge(df_result, mse_error)
df_result.to_csv("test_analysis_grp_by_dataset_wo_rank0.csv", index=False)
df_result

,Graph,Count,RE,MRE,MSE
0,graph_random_regular_graph_n7_d4,3786,268.283650,0.070862,0.082544
1,graph_powerlaw_cluster_graph_n7,1253,97.185105,0.077562,0.079149
2,star_graph_5_n7,11,1.497408,0.136128,0.106533
3,star_graph_n7,8,1.177959,0.147245,0.168330
4,star_graph_4_n7,6,0.934693,0.155782,0.197348
5,star_graph_6_n7,6,0.520716,0.086786,0.116281
6,star_graph_2_n7,5,0.602694,0.120539,0.179469
7,star_graph_1_n7,5,0.417483,0.083497,0.127023
8,star_graph_3_n7,4,0.384669,0.096167,0.130972


# Group by Dataset and Rank

In [13]:
groupby = ['Graph', 'Rank']

values_count, re_sum, mre_error, mse_error = group_by_table(df, groupby)

df_result = pd.merge(values_count, re_sum).sort_values(by=['Graph', 'Count'], ascending=[True, False])
df_result = pd.merge(df_result, mre_error)
df_result = pd.merge(df_result, mse_error)
df_result.to_csv("test_analysis_grp_by_dataset_rank.csv", index=False)
df_result

,Graph,Rank,Count,RE,MRE,MSE
0,graph_powerlaw_cluster_graph_n7,2.0,405,39.009171,0.096319,0.067697
1,graph_powerlaw_cluster_graph_n7,1.0,369,15.256954,0.041347,0.002848
2,graph_powerlaw_cluster_graph_n7,3.0,317,30.964299,0.097679,0.148509
3,graph_powerlaw_cluster_graph_n7,4.0,144,10.855049,0.075382,0.142035
4,graph_powerlaw_cluster_graph_n7,0.0,97,0.000000,0.000000,0.000000
5,graph_powerlaw_cluster_graph_n7,5.0,17,1.061502,0.062441,0.183711
6,graph_powerlaw_cluster_graph_n7,6.0,1,0.038130,0.038130,0.052340
7,graph_random_regular_graph_n7_d4,3.0,1393,118.630680,0.085162,0.122339
8,graph_random_regular_graph_n7_d4,2.0,1202,84.570780,0.070358,0.040838
9,graph_random_regular_graph_n7_d4,1.0,622,21.123545,0.033961,0.001507
